# Tweede kamer Topic modeling

In [1]:
import pandas as pd
import numpy as np
import gensim
import plotly.express as px
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

import umap

import spacy
nlp = spacy.load("nl")

# Gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel
import pyLDAvis.gensim

## Import data

In [ ]:
## put data chuncks in one zip file
#!cat data/CorpusTweedeKamer* > CorpusTweedeKamer.zip

In [2]:
%%time
tweede_kamer = pd.read_csv("CorpusTweedeKamer.zip")
tweede_kamer = (
    tweede_kamer
    .assign(datum = pd.to_datetime(tweede_kamer.date))
    .assign(speaker = tweede_kamer.speaker.str.lower())
)

CPU times: user 12.7 s, sys: 901 ms, total: 13.6 s
Wall time: 13.7 s


In [3]:
tweede_kamer.shape

(1143366, 12)

In [5]:
tweede_kamer.sample(10)

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,datum
659615,2011-04-21,NaN,362,zijlstra,VVD,828.0,False,21,Dat was mij niet ontgaan.,NL-TweedeKamer,NLD,2011-04-21
477094,2007-10-04,NaN,127,kant,SP,1363.0,False,53,Daarmee zijn wij weer terug bij de doelstellin...,NL-TweedeKamer,NLD,2007-10-04
103218,1997-06-19,NaN,23,oedayraj singh varma,GL,1537.0,False,51,De heer Van Boxtel noemt de ons omringende lan...,NL-TweedeKamer,NLD,1997-06-19
77785,1996-11-20,NaN,87,noorman-den uyl,PvdA,1234.0,False,1185,"Voorzitter! Ik vraag de minister in dit kader,...",NL-TweedeKamer,NLD,1996-11-20
743885,2012-12-11,NaN,341,kerstens,PvdA,1234.0,False,106,Dat zijn twee vragen. De eerste vraag dacht ik...,NL-TweedeKamer,NLD,2012-12-11
276049,2001-12-13,NaN,38,slob,CU,1459.0,False,219,Op uw eerste vraag antwoord ik heel nadrukkeli...,NL-TweedeKamer,NLD,2001-12-13
940225,2016-06-15,NaN,442,de heer bisschop,SGP,1178.0,False,119,Ik zou niet weten hoe de SGP-fractie in de Eer...,NL-TweedeKamer,NLD,2016-06-15
295218,2002-09-25,NaN,55,de graaf,D66,45.0,False,99,Voorzitter. Het zou niet in mij opkomen om de ...,NL-TweedeKamer,NLD,2002-09-25
360245,2004-06-03,NaN,208,schippers,VVD,828.0,False,21,Dat geldt ook voor de apotheken die soms in ei...,NL-TweedeKamer,NLD,2004-06-03
868394,2015-04-07,NaN,153,segers,CU,1459.0,False,144,De zorgen in 2008 waren als volgt. Is het prop...,NL-TweedeKamer,NLD,2015-04-07


## TOPIC Modeling

### selection of more recent speeches that are longer

In [3]:
recente_speeches = (
    tweede_kamer
    .query('datum > "2019-01-01"')
    .query('terms > 40')
    .query('terms < 1000')
)

In [5]:
recente_speeches

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,datum
1101585,2019-01-15,NaN,4,de heer klaver,GL,1537.0,False,327,Voorzitter. Als deze premier naar het grote be...,NL-TweedeKamer,NLD,2019-01-15
1101587,2019-01-15,NaN,6,minister rutte,NaN,NaN,False,485,"Voorzitter. Het doel, in de Klimaatwet, waarva...",NL-TweedeKamer,NLD,2019-01-15
1101588,2019-01-15,NaN,7,de heer klaver,GL,1537.0,False,276,Ik hoor hier randvoorwaarden. Dat kon je vorig...,NL-TweedeKamer,NLD,2019-01-15
1101589,2019-01-15,NaN,8,minister rutte,NaN,NaN,False,275,Op het tweede punt heb ik al geantwoord dat ik...,NL-TweedeKamer,NLD,2019-01-15
1101591,2019-01-15,NaN,10,de heer klaver,GL,1537.0,False,204,Om ervoor te zorgen dat er zo breed mogelijk d...,NL-TweedeKamer,NLD,2019-01-15
...,...,...,...,...,...,...,...,...,...,...,...,...
1143358,2019-07-04,NaN,962,de heer van ojik,GL,1537.0,False,65,Ik zou graag weten op basis waarvan de staatss...,NL-TweedeKamer,NLD,2019-07-04
1143359,2019-07-04,NaN,963,staatssecretaris broekers-knol,NaN,NaN,False,290,Ik ga uit van de gegevens die wij hebben gekre...,NL-TweedeKamer,NLD,2019-07-04
1143361,2019-07-04,NaN,965,staatssecretaris broekers-knol,NaN,NaN,False,142,Dan de motie op stuk nr. 2519 van de heer Hidd...,NL-TweedeKamer,NLD,2019-07-04
1143363,2019-07-04,NaN,967,de voorzitter,NaN,NaN,True,62,Over exact 60 minuten gaan wij stemmen over de...,NL-TweedeKamer,NLD,2019-07-04


### clean up

stopwords

digits

In [4]:
%%time 

from pprint import pprint
from collections import defaultdict

# remove common words and tokenize
nlstop = spacy.lang.nl.stop_words.STOP_WORDS

texts = [
    [word for word in document.lower().split() if word not in nlstop]
    for document in recente_speeches.text
]

CPU times: user 722 ms, sys: 50.5 ms, total: 773 ms
Wall time: 779 ms


### create dict and corpus

In [5]:
%%time 

dictionary = corpora.Dictionary(texts)
print(dictionary)

#### filter out extremes too little or too much frequencies.....
dictionary.filter_extremes(
    no_below = 5,
    no_above = 0.95
)
print(dictionary)

corpus = [dictionary.doc2bow(text) for text in texts]

Dictionary(102488 unique tokens: ['150', '2018', '2018,', '2019', '2030']...)
Dictionary(22785 unique tokens: ['150', '2018', '2018,', '2019', '2030']...)
CPU times: user 3.32 s, sys: 84 ms, total: 3.4 s
Wall time: 3.42 s


In [6]:
%%time
tweedekamer_lda = LdaModel(corpus, num_topics = 16, id2word = dictionary, passes = 10)

CPU times: user 2min 5s, sys: 943 ms, total: 2min 6s
Wall time: 2min 7s


In [7]:
lda_topics = tweedekamer_lda.print_topics(num_topics=16 )
lda_topics

[(0,
  '0.192*"zorg" + 0.069*"medische" + 0.032*"patiënten" + 0.027*"zorg," + 0.025*"patiënt" + 0.024*"zorg." + 0.015*"gegevens" + 0.013*"straffen" + 0.012*"zorgverzekeraars" + 0.012*"onafhankelijkheid"'),
 (1,
  '0.021*"kabinet" + 0.015*"gaat" + 0.015*"bedrijven" + 0.011*"geld" + 0.010*"grote" + 0.010*"gaan" + 0.009*"miljoen" + 0.008*"jaar" + 0.008*"natuur" + 0.007*"miljard"'),
 (2,
  '0.056*"nederland" + 0.054*"europese" + 0.035*"landen" + 0.025*"nederlandse" + 0.021*"commissie" + 0.018*"europa" + 0.015*"europees" + 0.011*"unie" + 0.011*"land" + 0.008*"internationale"'),
 (3,
  '0.247*"staatssecretaris" + 0.020*"staatssecretaris." + 0.014*"co2-heffing" + 0.013*"gevangenis" + 0.012*"asielzoekers" + 0.011*"staatssecretaris," + 0.009*"megaton" + 0.008*"cijfers" + 0.007*"optie" + 0.006*"afschaffen"'),
 (4,
  '0.059*"minister" + 0.030*"voorzitter." + 0.018*"dank" + 0.011*"vraag" + 0.009*"graag" + 0.008*"wel." + 0.007*"nederland" + 0.007*"wel," + 0.007*"vragen" + 0.006*"gaat"'),
 (5,
  '0.

In [16]:
pyLDAvis.enable_notebook(local = True)

In [8]:
%%time 

vis_data = pyLDAvis.gensim.prepare(tweedekamer_lda, corpus, dictionary)
pyLDAvis.show(vis_data)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8888/    [Ctrl-C to exit]
127.0.0.1 - - [07/Aug/2020 14:39:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Aug/2020 14:39:41] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [07/Aug/2020 14:39:41] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Aug/2020 14:39:41] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Aug/2020 14:39:41] code 404, message Not Found
127.0.0.1 - - [07/Aug/2020 14:39:41] "GET /favicon.ico HTTP/1.1" 404 -

stopping Server...
CPU times: user 23.1 s, sys: 350 ms, total: 23.5 s
Wall time: 52.5 s
